In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import src.tratamiento_datos as td

In [27]:
import requests
from bs4 import BeautifulSoup

In [3]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [5]:
companies = db.get_collection('companies')

In [6]:
companies.distinct('category_code')

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

nos hacemos una lista de todas las compañías quedándonos con su nombre, su categoría, el dinero conseguido y la lista de oficinas.

In [7]:
proj = {'_id' : 0, 'name' : 1, 'category_code' : 1, 'total_money_raised' : 1, 'offices' : 1}
companies_1m = list(companies.find({},proj))

In [8]:
companies_1m[0]['offices'][0]

{'description': '',
 'address1': '710 - 2nd Avenue',
 'address2': 'Suite 1100',
 'zip_code': '98104',
 'city': 'Seattle',
 'state_code': 'WA',
 'country_code': 'USA',
 'latitude': 47.603122,
 'longitude': -122.333253}

Limpiamos un poquito los datos para quedarnos únicamente con los que nos interesan.

In [9]:
td.limpiar_offices(companies_1m)

In [10]:
td.pasar_float(companies_1m)

In [11]:
companies_1m

[{'name': 'Wetpaint',
  'category_code': 'web',
  'total_money_raised': 39800000.0,
  'offices': [{'city': 'Seattle'}, {'city': 'New York'}]},
 {'name': 'AdventNet',
  'category_code': 'enterprise',
  'total_money_raised': '0',
  'offices': [{'city': 'Pleasanton'}]},
 {'name': 'Zoho',
  'category_code': 'software',
  'total_money_raised': '0',
  'offices': [{'city': 'Pleasanton'}]},
 {'name': 'Digg',
  'category_code': 'news',
  'total_money_raised': 45000000.0,
  'offices': [{'city': 'San Francisco'}]},
 {'name': 'Facebook',
  'category_code': 'social',
  'total_money_raised': 2430000000.0,
  'offices': [{'city': 'Menlo Park'},
   {'city': 'Dublin'},
   {'city': 'New York'}]},
 {'name': 'Omnidrive',
  'category_code': 'network_hosting',
  'total_money_raised': 800000.0,
  'offices': [{'city': 'Palo Alto'}]},
 {'name': 'Postini',
  'category_code': 'web',
  'total_money_raised': '0',
  'offices': [{'city': 'San Carlos'}]},
 {'name': 'Geni',
  'category_code': 'web',
  'total_money_rais

Eliminamos todas las empresas con un beneficio inferior a 1.000.000

In [13]:
lista_1m = td.limpiar_1m(companies_1m)

Eliminamos todas las empresas que no tienen que ver con el sector tech

In [14]:
lista_1m_sector = td.limpiar_sector(lista_1m)

Nos quedamos solo con las ciudades

In [21]:
lista_ciudades = td.seleccionar_ciudades(lista_1m_sector)

In [22]:
len(lista_ciudades)

2942

Y hacemos que la lista sea de valores únicos

In [24]:
lista_ciudades = set(lista_ciudades)

In [26]:
lista_ciudades

{'',
 '00134 Roma',
 '2314 XZ Leiden',
 '303 Twin Dolphin Drive Suite 600',
 '46395 Bocholt',
 '5-12-5 Ginza, Chuo-ku',
 'Aachen',
 'Abbotsford',
 'Abercynon',
 'Acton',
 'Addison',
 'Alberta',
 'Albuquerque',
 'Alcobendas',
 'Alexandria',
 'Aliso Viejo',
 'Almere',
 'Alpharetta',
 'Amherst',
 'Amman',
 'Amsterdam',
 'Andover',
 'Ann Arbor',
 'Annapolis',
 'Antwerp',
 'Arcadia',
 'Arlington',
 'Arlington Heights',
 'Aschheim-Dornach',
 'Asnieres sur seine',
 'Atlanta',
 'Atlanta,',
 'Auckland',
 'Austin',
 'BIDART',
 'BROOMFIELD',
 'Baar ZG',
 'Bahía Blanca',
 'Baltimore',
 'Banemarksvej 50 G',
 'Bangalore',
 'Bangkok',
 'Barcelona',
 'Bath',
 'Bath, Somerset',
 'Baton Rouge',
 'Beachwood',
 'Beaverton',
 'Beijing',
 'Belfast',
 'Bellevue',
 'Belmont',
 'Berkeley',
 'Berkshire',
 'Berlin',
 'Bet Shemesh',
 'Bethesda',
 'Beverly Hills',
 'Bilekahalli, Bangalore',
 'Billerica',
 'Birmingham',
 'Bloomfield',
 'Bluffdale',
 'Boca Raton',
 'Bogota',
 'Boothwyn',
 'Bordeaux',
 'Bordeaux Cede

Vamos a scrapear una tabla de ciudades con su coste de vida, la página elegida es https://es.numbeo.com/coste-de-vida/clasificaciones-actuales

In [29]:
url = 'https://es.numbeo.com/coste-de-vida/clasificaciones-actuales'
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')

In [45]:
cuerpo = soup.find_all('tbody')

In [51]:
filas = cuerpo[0].find_all('tr')

In [61]:
filas[0].text.strip('\n').split('\n')

['Hamilton, Islas Bermudas',
 '149,30',
 '98,19',
 '125,58',
 '147,70',
 '166,13',
 '80,79']

In [85]:
coste_vida_ciudades = {
    'ciudad' : [],
    'país' : [],
    'coste_vida' : [],
    'alquiler' : [],
    'coste_vida_alquiler' : [],
    'alimentos' : [], 
    'restaurantes' : [],
    'poder_adquisitivo_local' : []
}

In [86]:
for elem in filas:
    coste_vida_ciudades['coste_vida'].append(float(elem.text.strip('\n').split('\n')[1].replace(',', '.')))
    coste_vida_ciudades['alquiler'].append(float(elem.text.strip('\n').split('\n')[2].replace(',', '.')))
    coste_vida_ciudades['coste_vida_alquiler'].append(float(elem.text.strip('\n').split('\n')[3].replace(',', '.')))
    coste_vida_ciudades['alimentos'].append(float(elem.text.strip('\n').split('\n')[4].replace(',', '.')))
    coste_vida_ciudades['restaurantes'].append(float(elem.text.strip('\n').split('\n')[5].replace(',', '.')))
    coste_vida_ciudades['poder_adquisitivo_local'].append(float(elem.text.strip('\n').split('\n')[1].replace(',', '.')))
    

In [103]:
ciudades = [elem.text.strip('\n').split('\n')[0].split(',')[0].strip() for elem in filas]
paises = [elem.text.strip('\n').split('\n')[0].split(',')[1].strip() for elem in filas]

In [104]:
coste_vida_ciudades['ciudad'] = ciudades
coste_vida_ciudades['país'] = paises

In [105]:
coste_vida_ciudades

{'ciudad': ['Hamilton',
  'Zúrich',
  'Basilea',
  'Lausana',
  'Lugano',
  'Ginebra',
  'Berna',
  'Stavanger',
  'Tromso',
  'Oslo',
  'Trondheim',
  'Bergen',
  'Nueva York',
  'La Valeta',
  'Reikiavik',
  'Saint Helier',
  'Honolulu',
  'Santa Bárbara',
  'Copenhague',
  'Arhus',
  'San Francisco',
  'Tel Aviv-Yafo',
  'Anchorage',
  'París',
  'Oakland',
  'Beirut',
  'Aalborg',
  'Nasáu',
  'Luxemburgo',
  'Rejovot',
  'Sídney',
  'Darwin',
  'Queens',
  'Seattle',
  'Londres',
  'Boston',
  'Tokio',
  'Niza',
  'Auckland',
  'Jerusalén',
  'Dublín',
  'Perth',
  'Estocolmo',
  'Brisbane',
  'Galway',
  'Antibes',
  'Seúl',
  'Petaj Tikva',
  'Washington D. C.',
  'Singapur',
  'Turku',
  'Jersey City',
  'Ámsterdam',
  'Cork',
  'Haifa',
  'Odense',
  'Utrecht',
  'Brighton',
  'Upsala',
  'Canberra',
  'Múnich',
  'Osaka',
  'Melbourne',
  'Newcastle',
  'Dubrovnik',
  'Gold Coast',
  'Lyon',
  'Helsinki',
  'Gotemburgo',
  'Los Angeles',
  'Milán',
  'Breda',
  'Saint Paul',
